In [1]:
import numpy as np
import matplotlib.pyplot as plt
from typing import Type, Iterable

# import the neccesary pyquil modules
from qaoa.cost_function import QAOACostFunctionOnQVM, QAOACostFunctionOnWFSim
from pyquil.api import local_qvm, WavefunctionSimulator
from pyquil.paulis import PauliSum, PauliTerm

# import the QAOAParameters that we want to demo
from qaoa.parameters import GeneralQAOAParameters,QAOAParameterIterator

from vqe.optimizer import scipy_optimizer
import utilities as ut

This section shows how to use the QAOAParameterIterator class, which allows one to compute the cost function value of parametric wavefunctions produced by QAOA circuits, where one or two parameters of interest are varied, and all others are kept fixed. Let's work again with a simple 2-qubit problem, in the GeneralQAOAParameters class, with 𝑝=3 timesteps.

In [2]:
h_test = PauliSum.from_compact_str("0.7*Z0*Z1 + (-0.5)*Z0")

We require 2x3 betas parameters, 1x3 gammas_singles and 1x3 gammas_pairs parameters, which we will initialise randomly:

In [3]:
betas = np.random.rand(6)
gammas_singles = np.random.rand(3)
gammas_pairs = np.random.rand(3)
parameters = (betas,gammas_singles,gammas_pairs)

In [4]:
GeneralParams = GeneralQAOAParameters([h_test,3],parameters)
print(GeneralParams)

Hyperparameters:
	register: [0, 1]
	qubits_singles: [0]
	qubits_pairs: [[0, 1]]
Parameters:
	betas: [0.59861971 0.48487523 0.26523811 0.35101566 0.82798453 0.36307735]
	gammas_singles: [0.9509399  0.29840601 0.86025737]
	gammas_pairs: [0.19549544 0.46747086 0.36080255]



We have a total of 12 parameters; as an example, let's focus on how the cost function value changes when we vary the beta angle on the second qubit in the second QAOA step (p=2). This is the 4th entry (index 3) in the betas list. We will vary its range from 0 to 2𝜋.

In [6]:
range_ = np.linspace(0,2*np.pi,100)
param_ = "betas[4]"
iterator = QAOAParameterIterator(GeneralParams, param_, range_)

We now want to compute the cost function value with all the different values of the specified parameter of interest. 

In [7]:
sim = WavefunctionSimulator() # Define the wavefunction simulator

cost_vals = np.zeros((len(range_,)))
dev_vals = np.zeros((len(range_,)))
for p in iterator:

    print(p) ## IN THIS LINE p PRINTS AS EXPECTED
    cost_function = QAOACostFunctionOnWFSim(h_test,
                                            params=p,
                                            sim=sim,
                                            return_standard_deviation=True,
                                            noisy=False,
                                            log=[])
    
    
    out = cost_function(p.raw())
    print(out)
    print(p) ## BY THIS LINE, FOR SOME REASON THE PARAMETERS HAVE BEEN SLICED INTO 2-TUPLES FOR BETAS, 
             ## GAMMAS (SEE BELOW), GIVING AN ERROR IN THE NEXT ITERATION

Hyperparameters:
	register: [0, 1]
	qubits_singles: [0]
	qubits_pairs: [[0, 1]]
Parameters:
	betas: [0.59861971 0.48487523 0.26523811 0.35101566 0.         0.36307735]
	gammas_singles: [0.9509399  0.29840601 0.86025737]
	gammas_pairs: [0.19549544 0.46747086 0.36080255]

(-0.857163243316778, 0.014953097626452357)
Hyperparameters:
	register: [0, 1]
	qubits_singles: [0]
	qubits_pairs: [[0, 1]]
Parameters:
	betas: [[0.59861971 0.48487523], [0.26523811 0.35101566], [0.         0.36307735]]
	gammas_singles: [[0.9509399 ], [0.29840601], [0.86025737]]
	gammas_pairs: [[0.19549544], [0.46747086], [0.36080255]]



IndexError: index 4 is out of bounds for axis 0 with size 3